In [1]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn import linear_model


In [2]:
os.getcwd()
os.chdir(projectDir)

In [3]:
adataSingle = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_08122020_PerGENE.h5ad')
adataSingle.obs['GENE_INEFFECT_'] = 0
adataSingle.uns['feature_barcode_names_filtered_GENES'] = [ x for x in adataSingle.obs.columns if (x.startswith("GENE_"))]


In [4]:
adataMultiple = sc.read('outputs/anndata/adata-hash-features_singlets_MultipleKO_06292020_PerGENE.h5ad')

In [5]:
adataSingle.X.shape

(242938, 6685)

In [6]:
adataMultiple.X.shape

(77873, 6685)

In [7]:
genesB = adataSingle.uns['feature_barcode_names_filtered_GENES']
genesB

['GENE_4921501E09Rik_',
 'GENE_4930595M18Rik_',
 'GENE_8030462N17Rik_',
 'GENE_Aamp_',
 'GENE_Abtb1_',
 'GENE_Abtb2_',
 'GENE_Acaca_',
 'GENE_Ago2_',
 'GENE_Ahctf1_',
 'GENE_Ahr_',
 'GENE_Akt1_',
 'GENE_Ambra1_',
 'GENE_Amfr_',
 'GENE_Anapc10_',
 'GENE_Anapc11_',
 'GENE_Anapc13_',
 'GENE_Anapc1_',
 'GENE_Anapc2_',
 'GENE_Anapc4_',
 'GENE_Anapc5_',
 'GENE_Anapc7_',
 'GENE_Ankfy1_',
 'GENE_Apaf1_',
 'GENE_Apbb1_',
 'GENE_Apc_',
 'GENE_Ar_',
 'GENE_Arel1_',
 'GENE_Arih1_',
 'GENE_Arih2_',
 'GENE_Armc8_',
 'GENE_Arnt_',
 'GENE_Arpc1a_',
 'GENE_Arpc1b_',
 'GENE_Arrb1_',
 'GENE_Asb11_',
 'GENE_Asb12_',
 'GENE_Asb14_',
 'GENE_Asb15_',
 'GENE_Asb1_',
 'GENE_Asb2_',
 'GENE_Asb3_',
 'GENE_Asb4_',
 'GENE_Asb6_',
 'GENE_Asb7_',
 'GENE_Asb9_',
 'GENE_Atg16l1_',
 'GENE_Atg16l2_',
 'GENE_Atg5_',
 'GENE_Atxn3_',
 'GENE_Aurka_',
 'GENE_Bach1_',
 'GENE_Bard1_',
 'GENE_Baz1a_',
 'GENE_Baz1b_',
 'GENE_Bcl6_',
 'GENE_Bfar_',
 'GENE_Bid_',
 'GENE_Birc2_',
 'GENE_Birc3_',
 'GENE_Bmi1_',
 'GENE_Bop1_',
 'GENE

In [8]:
print(np.setdiff1d(adataMultiple.uns['feature_barcode_names_filtered_GENES'], adataSingle.uns['feature_barcode_names_filtered_GENES']))


[]


In [9]:
len(adataMultiple.uns['feature_barcode_names_filtered_GENES'])


1033

In [10]:
len(adataSingle.uns['feature_barcode_names_filtered_GENES'])

1033

In [11]:
k = adataSingle.obs[adataSingle.uns['feature_barcode_names_filtered_GENES']]
k.sum(axis=1).value_counts()

1    242938
dtype: int64

In [12]:
k = adataMultiple.obs[adataMultiple.uns['feature_barcode_names_filtered_GENES']]
k.sum(axis=1).value_counts()

2    68137
3     9266
4      464
5        6
dtype: int64

In [13]:
len(adataMultiple.uns['feature_barcode_names_filtered'])

2561

In [14]:
len(adataSingle.uns['feature_barcode_names_filtered'])

2561

In [15]:
adataAll = adataSingle.concatenate(adataMultiple)

In [16]:
adataAll.uns['feature_barcode_names_filtered_GENES'] = adataSingle.uns['feature_barcode_names_filtered_GENES']
adataAll.uns['feature_barcode_names_filtered'] = adataSingle.uns['feature_barcode_names_filtered']
adataAll.obs[adataAll.uns['feature_barcode_names_filtered_GENES']].shape

(320811, 1033)

In [17]:
adataAll.X.shape

(320811, 6685)

In [18]:
adataAll.layers['counts']

<320811x6685 sparse matrix of type '<class 'numpy.float32'>'
	with 568775082 stored elements in Compressed Sparse Row format>

In [19]:
len(adataAll.uns['feature_barcode_names_filtered'])

2561

In [20]:
adataAll.obs["SubCellType_0"] = 0
adataAll.obs.loc[adataAll.obs.leiden.isin(['0','1','4','7']),"SubCellType_0"] = 1
adataAll.obs["SubCellType_1"] = 0
adataAll.obs.loc[adataAll.obs.leiden.isin(['2','6']),"SubCellType_1"] = 1
adataAll.obs["SubCellType_2"] = 0
adataAll.obs.loc[adataAll.obs.leiden.isin(['5','8']),"SubCellType_2"] = 1
adataAll.obs["SubCellType_3"] = 0
adataAll.obs.loc[adataAll.obs.leiden.isin(['3','9']),"SubCellType_3"] = 1


In [21]:
adataAll.write('outputs/anndata/adata-hash-features_ALLKO_11152020_PerGENE.h5ad')

... storing 'KOType' as categorical
... storing 'Cell_category' as categorical


In [ ]:
adataAll.X.shape

In [ ]:
signLMCoef = pd.read_csv('R/SignificantBetaCoefs.csv', index_col=0)
signLMCoef

In [ ]:
adataSel = adataAll[:,signLMCoef.columns].copy()

In [ ]:
adataSel.X.shape

In [ ]:
adataSel.uns['feature_barcode_names_filtered_GENES'] = genesB

In [ ]:
adataSel.obs[adataSel.uns['feature_barcode_names_filtered_GENES']].shape

In [ ]:
adataSel.X.shape

In [ ]:
df = adataSel.obs[["n_genes", "mt_frac", "leiden" ]]
dummies = pd.get_dummies(df.leiden)
df = df.join(dummies)
df.drop(["leiden"], axis=1,inplace = True) 

rnaMat = adataSel.X
rnaMat = pd.DataFrame(rnaMat)
rnaMat.columns = adataSel.var_names
rnaMat.index = adataSel.obs_names

regr = linear_model.LinearRegression(fit_intercept = False) # Do not use fit_intercept = False if you have removed 1 column after dummy encoding
regr.fit(df, rnaMat)
predicted = regr.predict(df)

residuals = rnaMat-predicted

adataSel.layers["ClusterResiduals"] = residuals


In [ ]:
adataSel.layers["ClusterResiduals"]


In [ ]:
adataSel.write('outputs/anndata/adata-hash-features_ALLKO_08222020_PerGENE_SelectedGenes.h5ad')